<a href="https://colab.research.google.com/github/tuanchris/auto-image-tagging/blob/master/Auto_image_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pwd

/content


In [3]:
%cd /content/gdrive/My\ Drive/projects/auto-image-tagging
# !git clone https://github.com/tuanchris/auto-image-tagging

/content/gdrive/My Drive/projects/auto-image-tagging


In [0]:
import os
os.chdir('/content/gdrive/My Drive/projects/auto-image-tagging')

In [5]:
!python3 ./src/keras_json_parser.py --json_file ./src/labels.json --dataset_path ./data/ --train_percentage 80 --validation_percentage 20

dict_keys(['day', 'indoor', 'water', 'mountain', 'human', 'tree', 'object', 'building', 'car', 'animal', 'night', 'road', 'trash', 'boat', 'doc', 'plane'])
Traceback (most recent call last):
  File "./src/keras_json_parser.py", line 59, in <module>
    os.mkdir(args.dataset_path)
FileExistsError: [Errno 17] File exists: './data/'


In [6]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint



Using TensorFlow backend.


In [7]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './data/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        './data/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 492 images belonging to 16 classes.
Found 123 images belonging to 16 classes.


In [8]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(16,activation='sigmoid')(x)


# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer=sgd, metrics = ['accuracy'])

filepath="./models/weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.summary()
















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalizati

In [9]:
# train the model on the new data for a few epochs
# model.fit_generator(
#         train_generator,
#         steps_per_epoch=2000,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=800)

H = model.fit_generator(
	train_generator,
	validation_data=validation_generator,
	steps_per_epoch=2000,
	epochs=50, verbose=1, 
	callbacks=callbacks_list)




Epoch 1/50
2000/2000 [==============================] - 454s 227ms/step - loss: 0.1259 - acc: 0.9560 - val_loss: 0.2274 - val_acc: 0.9441

Epoch 00001: val_acc improved from -inf to 0.94411, saving model to ./models/weights-improvement-01.hdf5
Epoch 2/50
2000/2000 [==============================] - 438s 219ms/step - loss: 0.0722 - acc: 0.9724 - val_loss: 0.2710 - val_acc: 0.9400

Epoch 00002: val_acc did not improve from 0.94411
Epoch 3/50
2000/2000 [==============================] - 450s 225ms/step - loss: 0.0577 - acc: 0.9780 - val_loss: 0.2894 - val_acc: 0.9421

Epoch 00003: val_acc did not improve from 0.94411
Epoch 4/50
2000/2000 [==============================] - 442s 221ms/step - loss: 0.0505 - acc: 0.9811 - val_loss: 0.3086 - val_acc: 0.9431

Epoch 00004: val_acc did not improve from 0.94411
Epoch 5/50
2000/2000 [==============================] - 441s 220ms/step - loss: 0.0442 - acc: 0.9838 - val_loss: 0.3303 - val_acc: 0.9411

Epoch 00005: val_acc did not improve from 0.9441